In [1]:
# Data Handlers
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps

# Pytorch
import torch
import torch.nn as nn  # NN; networks (CNN, RNN, losses)
import torch.optim as optim  # Optimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, IterableDataset  # Dataset manager
# from torch.autograd import Variable
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

# Other
import datetime
import os
from tqdm import tqdm
from pathlib import Path
from os import listdir
from skimage.metrics import structural_similarity as ssim
# from fastaniso import anisodiff
# from time import time
from IPython.display import clear_output

# Graphics
from matplotlib import pyplot as plt
import seaborn as sns

# Additional modules
# from dataset_creator import generate_csv
# from assistive_funcs import filtering_image, check_ssim, convert_to_grayscale, get_dataset_name
# from csv_dataloader import get_train_test_data
from math import floor


In [2]:
# Paths
p_main_data = Path("../data")
p_models = Path("../models")

p_scv_folder = p_main_data / "csv_files"  # datasets_path
p_img = p_main_data / "images"

p_noised_imgs = p_main_data / "imgs_with_noise"

p_filtered_images = p_main_data / "filtered_imgs"

p_gray_images = p_main_data / "gray_images"

# Dataset
win_size = 7
_step = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device = }")


device = device(type='cuda')


In [3]:
def add_noise(img, scale=0.2707) -> np.array:

    # Create noise
    size = img.shape
    noise = np.random.rayleigh(scale=scale, size=size)
    # Add noise
    noised_img = img + img * noise
    noised_img = np.where(noised_img <= 255, noised_img, 255)
    return torch.Tensor(noised_img)


In [4]:
def load_data(w, h, test_size=0.2, batch_size=32):

    transform = transforms.Compose([
        transforms.Resize(size=(w, h)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ])

    Data = datasets.ImageFolder(p_main_data / "images_for_cnn", transform=transform)

    num_data = len(Data)
    indices_data = list(range(num_data))
    np.random.shuffle(indices_data)
    split_tt = int(np.floor(test_size * num_data))
    train_idx, test_idx = indices_data[split_tt:], indices_data[:split_tt]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(
        Data, sampler=train_sampler, batch_size=batch_size)  # num_workers=4
    testloader = torch.utils.data.DataLoader(
        Data, sampler=test_sampler,  batch_size=batch_size)  # num_workers=4

    return trainloader, testloader


In [5]:
class Autoenc(nn.Module):
    def __init__(self, *, num_filter=64, num_channel=1, kernel_size=5, padding=1):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, num_filter, kernel_size, padding),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filter, num_filter, kernel_size, padding)
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(num_filter, 1, kernel_size, padding),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = nn.functional.interpolate(encoded, scale_factor=2)
        return self.decoder(decoded)


In [6]:
def test_train_loop(model, loaders: dict, epochs=3):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    accuracy = {"train": [], "test": []}
    for epoch in tqdm(range(epochs)):
        for stage, dataloader in loaders.items():
            for x_batch, y_batch in dataloader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device).to(torch.float32)

                if stage == "train":
                    model.train()
                    optimizer.zero_grad()
                    output = model(x_batch)

                    loss = criterion(output, y_batch)
                    loss.backward()
                    optimizer.step()

                    accuracy[stage].append(loss)

                else:
                    model.eval()
                    with torch.no_grad():
                        output = model(x_batch)
                        loss = criterion(output, y_batch)
                        accuracy[stage].append(loss)


In [7]:
w, h = 608, 608
batch_size = 1
test_size = 0.2
EPOCHS = 3

num_filter = 128

trainloader, testloader = load_data(w, h, test_size=test_size, batch_size=batch_size)

loaders = {"train": trainloader, "test": testloader}
model = Autoenc(num_filter=128).to(device)
test_train_loop(model, loaders, EPOCHS)

  0%|          | 0/3 [00:00<?, ?it/s]d:\Projects\PythonProjects\_venvs\ml_venv_p39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1, 592, 592])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 3/3 [00:18<00:00,  6.17s/it]
